In [70]:
#import dependencies
import pandas as pd
import os

In [71]:
#import csv into a df for use with Pandas
filepath = os.path.join("Resources","purchase_data.csv")
sourcedf = pd.read_csv(filepath, encoding = "UTF-8")

In [72]:
#review dataset
#see overview
sourcedf

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19


In [73]:
#list all columns
sourcedf.columns

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'], dtype='object')

In [74]:
#list data types per column
sourcedf.dtypes

Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

In [75]:
#Check for null values
sourcedf.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [76]:
#Look to see if consolidation is necessary in "Gender" values
genderList = sourcedf["Gender"].unique()
genderList
#Confirmed that all values are exclusive of each other

array(['Male', 'Other / Non-Disclosed', 'Female'], dtype=object)

In [77]:
#Data review: 
    #780 rows × 7 columns
    #Source data is a register of transactions. Each row is a uniqe transaction 
    #Unique player IDs are in col "SN"
    #Purchase ID column matches index no.
    #No null values present
    #All data types OK
    #No extraneous label values in "Gender"
#OK to proceed to calculations

In [78]:
#Player count
#get count of all unique players. First store single value into variable
playerCount = sourcedf["SN"].nunique()
#place variable into dataframe for display
playerCountdf = pd.DataFrame({"Total Players":[playerCount]})
playerCountdf

,Total Players
0,576


In [79]:
#Purchasing Analysis (Total)
#Get all values first and assign to variables, then assign variables into dataframe
#Get count of all unique items
itemCount = sourcedf["Item ID"].nunique()
#Get average of Price column
averagePurchPrice = sourcedf["Price"].mean()
#Get Total Number of Purchases as a row count of Purchase ID
numberOfPurchases = sourcedf["Purchase ID"].count()
#Get Total Revenue as a sum of all values in Price
totalRev = sourcedf["Price"].sum()

#place variables into dataframe
purchSummary = pd.DataFrame({"No. of Unique Items":[itemCount], 
                             "Avg. Purchase Price":[averagePurchPrice],
                             "Total Number of Purchases":[numberOfPurchases],
                             "Total Revenue":[totalRev]
                            })

#Format as necessary with .map()
purchSummary["Avg. Purchase Price"]=purchSummary["Avg. Purchase Price"].map("${:,.2f}".format)
purchSummary["Total Revenue"]=purchSummary["Total Revenue"].map("${:,.2f}".format)
#display summary dataframe
purchSummary

,No. of Unique Items,Avg. Purchase Price,Total Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [80]:
#Gender Demographics
#use playerCount for total player count

#Percentage and Count of Male Players
#find male players. Filter out male players players by gender
malePlayersdf = sourcedf.loc[sourcedf["Gender"] == "Male","SN"]
#Get unique count name of players
malePlayersCount = malePlayersdf.nunique()
#Get percentage of male players
malePercentage = malePlayersCount/playerCount

#Percentage and Count of Female Players
#find female players. Filter out female players players by gender
femalePlayersdf = sourcedf.loc[sourcedf["Gender"] == "Female","SN"]
#Get unique count name of players
femalePlayersCount = femalePlayersdf.nunique()
#Get percentage of male players
femalePercentage = femalePlayersCount/playerCount


#Percentage and Count of Other / Non-Disclosed
#find male players. Filter out male players players by gender
otherPlayersdf = sourcedf.loc[sourcedf["Gender"] == "Other / Non-Disclosed","SN"]
#Get unique count name of players
otherPlayersCount = otherPlayersdf.nunique()
#Get percentage of male players
otherPercentage = otherPlayersCount/playerCount


#Assemble summary table
genderDemoSummary = pd.DataFrame({"Total Count":[malePlayersCount, femalePlayersCount, otherPlayersCount], 
                             "Percentage of Players":[malePercentage, femalePercentage, otherPercentage]
                            })

#Set index labels using list
genderDemoSummary.index = ["Male","Female","Other / Non-Disclosed"]


#format percentages
#multiply by 100 to compensate for finding percentages from arithmetic
genderDemoSummary["Percentage of Players"] = genderDemoSummary["Percentage of Players"] * 100
#format
genderDemoSummary["Percentage of Players"] = genderDemoSummary["Percentage of Players"].map("{:,.2f}%".format)
genderDemoSummary

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [81]:
#Purchasing Analysis (Gender)
genderGroupdf= sourcedf.groupby(["Gender"])

#Find Purchase Count with .count() on any col
genderGroupCountdf = genderGroupdf["Purchase ID"].count()

#Average Purchase Price with .mean on "Price"
genderGroupAvgdf = genderGroupdf["Price"].mean()

#Total Purchase Value with .sum on "Price"
genderGroupSumdf = genderGroupdf["Price"].sum()

#Average Purchase Total per Person by Gender by dividing genderGroupSumdf series by unique player--
#counts by gender
genderAvgPerPersonByGenderdf = genderGroupSumdf/[femalePlayersCount, malePlayersCount, otherPlayersCount]

#Create summary table
purchAnalysisSummary = pd.DataFrame({"Purchase Count":genderGroupCountdf,
                                     "Avg Purchase Price":genderGroupAvgdf,
                                     "Total Purchase Value":genderGroupSumdf,
                                     "Avg Total Purchase per Person":genderAvgPerPersonByGenderdf
                            })
#Format summary table
purchAnalysisSummary["Avg Purchase Price"] = purchAnalysisSummary["Avg Purchase Price"].map("${:,.2f}".format)
purchAnalysisSummary["Total Purchase Value"] = purchAnalysisSummary["Total Purchase Value"].map("${:,.2f}".format)
purchAnalysisSummary["Avg Total Purchase per Person"] = \
purchAnalysisSummary["Avg Total Purchase per Person"].map("${:,.2f}".format)

#Display summary table
purchAnalysisSummary


,Purchase Count,Avg Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [82]:
#Age Demographics
#The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.) 	
    #find max age
sourcedf["Age"].max()




45

In [83]:
#set up bins
binVals = [0, 9.999, 14, 19, 24, 29, 34, 39, 49]
binLabels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#use .cut to break up into bins
sourcedf["Age Ranges"] = pd.cut(sourcedf["Age"], binVals, labels = binLabels)

#move Age Ranges to be the index
sourcedfAgeIndex = sourcedf.set_index("Age Ranges")

#group by age
sourcedfAgeIndexGroup = sourcedfAgeIndex.groupby(["Age Ranges"])

#Find player count of each age group
#Find unique players of each age group
sourcedfAgeUniquePlayers = sourcedfAgeIndexGroup["SN"].nunique()

#Find percentages
agePercentages = sourcedfAgeUniquePlayers/playerCount * 100

#Create age demo summary table
ageDemoSummary = pd.DataFrame({"Total Count":sourcedfAgeUniquePlayers,
                              "Percentage of Players":agePercentages})

#Format age demo summary table
ageDemoSummary["Percentage of Players"] = ageDemoSummary["Percentage of Players"].map("{:,.2f}%".format)
ageDemoSummary

,Total Count,Percentage of Players
Age Ranges,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [84]:
#Purchasing Analysis (Age)

#group by age
sourcedfAgeIndexGroup= sourcedfAgeIndex.groupby(["Age Ranges"])

#Find Purchase Count with .count() on Purchase ID
sourcedfAgeIndexCount = sourcedfAgeIndexGroup["Purchase ID"].count()

#Find Average Purchase Price with .mean() on Price
sourcedfAgeAvg = sourcedfAgeIndexGroup["Price"].mean()    

#Find Total Purchase Value with .sum() on Price
sourcedfAgeSum = sourcedfAgeIndexGroup["Price"].sum()

#Average Purchase Total per Person by Age Group by dividing genderGroupSumdf series by number of unique--
#buyers per age group
#Find unique SN's per age group
sourcedfAgeUniques = sourcedfAgeIndexGroup["SN"].nunique()
avgPurchaseTotalAge = sourcedfAgeSum/sourcedfAgeUniques

#Create summary table
ageSummary = pd.DataFrame({"Purchase Count":sourcedfAgeIndexCount,
                                     "Avg Purchase Price":sourcedfAgeAvg,
                                     "Total Purchase Value":sourcedfAgeSum,
                                     "Avg Total Purchase per Person":avgPurchaseTotalAge
                            })
#Format summary table
ageSummary["Avg Purchase Price"] = ageSummary["Avg Purchase Price"].map("${:,.2f}".format)
ageSummary["Total Purchase Value"] = ageSummary["Total Purchase Value"].map("${:,.2f}".format)
ageSummary["Avg Total Purchase per Person"] = \
ageSummary["Avg Total Purchase per Person"].map("${:,.2f}".format)

#Display summary table
ageSummary




,Purchase Count,Avg Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [85]:
user_total = sourcedf.groupby(["SN"]).sum()["Price"].rename("Total Purchase Value")
user_average = sourcedf.groupby(["SN"]).mean()["Price"].rename("Average Purchase Price")
user_count = sourcedf.groupby(["SN"]).count()["Price"].rename("Purchase Count")

# Convert to DataFrame
user_data = pd.DataFrame({"Total Purchase Value": user_total, "Average Purchase Price": user_average, "Purchase Count": user_count})

# Display Table
user_sorted = user_data.sort_values("Total Purchase Value", ascending=False)

# Minor Data Munging
user_sorted["Average Purchase Price"] = user_sorted["Average Purchase Price"].map("${:,.2f}".format)
user_sorted["Total Purchase Value"] = user_sorted["Total Purchase Value"].map("${:,.2f}".format)
user_sorted = user_sorted.loc[:,["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

# Display DataFrame
user_sorted.head(5)


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [86]:
#Top Spenders

#Group transactions by purchaser
playerGroup = sourcedf.groupby(["SN"])

#Find Purchase Count with .count
playerGroupCount = playerGroup["Purchase ID"].count()

#Find Average purchase price with .mean "Price"
playerGroupAvg = playerGroup["Price"].mean()

#Find Total Purchased for each player with .sum on "Price"
playerGroupSum = playerGroup["Price"].sum()



#build Summary table
#Create summary table
playerSummary = pd.DataFrame({"Purchase Count":playerGroupCount,
                                     "Avg Purchase Price":playerGroupAvg,
                                     "Total Purchase Value":playerGroupSum
                            })

#Sort by decreasing by "Total Purchase Value" with .sort_values
sortPlayerSummary = playerSummary.sort_values("Total Purchase Value", ascending=False)

#Format summary table
sortPlayerSummary["Avg Purchase Price"] = sortPlayerSummary["Avg Purchase Price"].map("${:.2f}".format)
sortPlayerSummary["Total Purchase Value"] = sortPlayerSummary["Total Purchase Value"].map("${:.2f}".format)

#Get the top 5 spenders
#pull first 5 with .iloc
sortPlayerSummaryTop = sortPlayerSummary.iloc[0:5,]
sortPlayerSummaryTop



,Purchase Count,Avg Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [87]:
#Most Popular Items
#retrieve Dataframe. Go over the HW example steps for each phase

#Group transactions by purchaser
itemGroup = sourcedf.groupby(["Item ID","Item Name"])


#Find Purchase Count with .count
itemGroupCount = itemGroup["Purchase ID"].count()

#Get Item Prices
itemGroupPrices = itemGroup["Price"].mean()

#Find Total Purchased for each player with .sum on "Price"
itemGroupSum = itemGroup["Price"].sum()


#build Summary table
#Create summary table
itemSummary = pd.DataFrame({"Purchase Count":itemGroupCount,
                                     "Item Price":itemGroupPrices,
                                     "Total Purchase Value":itemGroupSum
                            })


#Sort by decreasing by "Total Purchase Value" with .sort_values
sortItemSummary = itemSummary.sort_values("Total Purchase Value", ascending=False)
sortItemSummary
#Format summary table
#sortItemSummary["Item Price"] = sortItemSummary["Item Price"].map("${:.2f}".format)
#sortItemSummary["Total Purchase Value"] = sortItemSummary["Total Purchase Value"].map("${:.2f}".format)

#Get the top 5 spenders
#pull first 5 with .iloc
#sortItemSummaryTop = sortItemSummary.iloc[0:5,]
#sortItemSummaryTop




,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,4.614615,59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.230000,50.76
82,Nirvana,9,4.900000,44.10
145,Fiery Glass Crusader,9,4.580000,41.22
103,Singed Scalpel,8,4.350000,34.80
...,...,...,...,...
28,"Flux, Destroyer of Due Diligence",2,1.060000,2.12
125,Whistling Mithril Warblade,2,1.000000,2.00
126,Exiled Mithril Longsword,1,2.000000,2.00


In [88]:
#Most Profitable Items

